
# Sensitivity

This is an example of how to use the double machine learning approach for conditional value-at-risk estimation.
The code demonstrates the usage of the `DoubleMLCateEstimator` class and provides a step-by-step guide.


# Python: Sensitivity Analysis

This notebook illustrates the sensitivity analysis tools with the partiallly linear regression model (PLR). <br>
The DoubleML package implements sensitivity analysis based on [Chernozhukov et al. (2022)](https://www.nber.org/papers/w30302).



## Simulation Example

For illustration purposes, we will work with generated data. This enables us to set the counfounding strength, such that we can correctly access quality of e.g. the robustness values.



In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import doubleml as dml
from doubleml.datasets import make_confounded_plr_data, fetch_401K

### Data

The data will be generated via `make_confounded_plr_data` and set the confounding values to `cf_y=0.1` and `cf_d=0.1`.

Both parameters determine the strength of the confounding

- `cf_y` measures the proportion of residual variance in the outcome explained by unobserved confounders
- `cf_d` measires the porportion of residual variance of the treatment representer generated by unobserved confounders.

For further details, see [Chernozhukov et al. (2022)](https://www.nber.org/papers/w30302) or [User guide](https://docs.doubleml.org/stable/guide/guide.html).
Further, the true treatment effect is set to `theta=5.0`.



In [ ]:
cf_y = 0.1
cf_d = 0.1
theta = 5.0

In [ ]:
np.random.seed(42)
dpg_dict = make_confounded_plr_data(n_obs=1000, cf_y=cf_y, cf_d=cf_d, theta=theta)
x_cols = [f'X{i + 1}' for i in np.arange(dpg_dict['x'].shape[1])]
df = pd.DataFrame(np.column_stack((dpg_dict['x'], dpg_dict['y'], dpg_dict['d'])), columns=x_cols + ['y', 'd'])
dml_data = dml.DoubleMLData(df, 'y', 'd')
print(dml_data)

### DoubleML Object

We fit a `DoubleMLPLR` object and use basic random forest to flexibly estimate the nuisance elements.



In [ ]:
np.random.seed(42)
dml_obj = dml.DoubleMLPLR(dml_data,
                          ml_l=RandomForestRegressor(),
                          ml_m=RandomForestRegressor(),
                          n_folds=5,
                          score='partialling out')
dml_obj.fit()
print(dml_obj)

The effect estimate is biased due to the unobserved confounding and the corresponding confidence interval does not cover the true parameter `theta=5.0`



In [ ]:
dml_obj.confint()

### Sensitivity Analysis

To perform a sensitivity analysis with the [DoubleML](https://docs.doubleml.org/stable/index.html) package you can use the `sensitivity_analysis()` method. <br>
The sensitivity analysis is based on the strength of the confounding `cf_y` and `cf_d` (default values $0.03$) and the parameter `rho`, which measures the correlation between the difference of the long and short form of the outcome regression and the Riesz representer (the default value $1.0$ is conservative and considers adversarial counfounding). To additionally incorporate statistical uncertainty, a significance level (default $0.95$) is used.

These input parameters are used to calculate upper and lower bounds (including the corresponding confidence level) on the treatment effect estimate. 

Further, the sensitivity analysis includes a null hypothesis (default $0.0$), which is used to compute robustness values. The robustness value $RV$ is defined as the required confounding strength (`cf_y=rv` and `cf_d=rv`), such that the lower or upper bound of the causal parameter includes the null hypothesis. The robustness value $RVa$ defined analogous but additionally incorporates statistical uncertainty (as it is based on the confidence intervals of the bounds). For a more detailed explanation
see the [User Guide](https://docs.doubleml.org/stable/guide/guide.html) or [Chernozhukov et al. (2022)](https://www.nber.org/papers/w30302).

The results of the analysis can be printed via the `sensitivity_summary` property or directly accessed via the `sensitivity_params` property.



In [ ]:
dml_obj.sensitivity_analysis()
print(dml_obj.sensitivity_summary)

In [ ]:
dml_obj.sensitivity_params

The robustness value $RV$ means that unobserved counfounders would have to account for $34\%$ of the residual variation (in treatment and outcome) to explain away the treatment effect.

We can also consider a contour plot to consider different values of confounding `cf_y` and `cf_d`. The contour plot and robustness values are based on the upper or lower bounds based on the null hypothesis (in this case this results in the lower bound).

Remark that both, the robustness values and the contour plot use the prespecified value of `rho` and the ``null_hypothesis``.



In [ ]:
dml_obj.sensitivity_plot()

To consider a different null hypothesis and add a different scenario to the the contour plot, we can adjust the parameter `null_hypothesis`.



In [ ]:
dml_obj.sensitivity_analysis(cf_y=cf_y, cf_d=cf_d, null_hypothesis=theta)
print(dml_obj.sensitivity_summary)

In [ ]:
dml_obj.sensitivity_plot()

## Application: 401(k)

In this real-data example, we illustrate how to use the sensitivity analysis from the [DoubleML](https://docs.doubleml.org/stable/index.html) package to evaluate effect estimates from 401(k) eligibility on accumulated assets. The 401(k) data set has been analyzed in several studies, among others [Chernozhukov et al. (2018)](https://arxiv.org/abs/1608.00060), see [Kallus et al. (2019)](https://arxiv.org/abs/1912.12945) for quantile effects.

**Remark:**
This notebook focuses on sensitivity analysis. For a basic introduction to the [DoubleML](https://docs.doubleml.org/stable/index.html) package and a detailed example of the average treatment effect estimation for the 401(k) data set, we refer to the notebook [Python: Impact of 401(k) on Financial Wealth](https://docs.doubleml.org/stable/examples/py_double_ml_pension.html).



### Data and Effect Estimation

Define eligiblity as treatment and the net financial assets as outcome to construct a DoubleML object.



In [ ]:
data = fetch_401K(return_type='DataFrame')

# Set up basic model: Specify variables for data-backend
features_base = ['age', 'inc', 'educ', 'fsize', 'marr',
                 'twoearn', 'db', 'pira', 'hown']

# Initialize DoubleMLData (data-backend of DoubleML)
data_dml = dml.DoubleMLData(data,
                            y_col='net_tfa',
                            d_cols='e401',
                            x_cols=features_base)
print(data_dml)

Use basic random forests to estimate the nuisance elements and fit a [DoubleMLPLR](https://docs.doubleml.org/stable/api/generated/doubleml.DoubleMLPLR.html) model.



In [ ]:
learner_l = RandomForestRegressor(n_estimators=500, max_depth=7,
                                  max_features=3, min_samples_leaf=3)
learner_m = RandomForestClassifier(n_estimators=500, max_depth=5,
                                   max_features=4, min_samples_leaf=7)

In [ ]:
np.random.seed(42)
dml_plr_obj = dml.DoubleMLPLR(data_dml,
                              ml_l = learner_l,
                              ml_m = learner_m,
                              n_folds = 5)
dml_plr_obj.fit()
print(dml_plr_obj)

### Sensitivity Analysis

In their paper [Chernozhukov et al. (2022)](https://www.nber.org/papers/w30302) argue that confounding should not account for more than $4\%$ of the residual variation of the outcome and $3\%$ of the residual variation of the treatment. Consequently, we set `cf_y=0.04` and `cf_d=0.03`. 



In [ ]:
dml_plr_obj.sensitivity_analysis(cf_y=0.04, cf_d=0.03)
print(dml_plr_obj.sensitivity_summary)

Our robustness values are similar to the ones in the paper. The effect estimate is robust in the sense that unobserved confounders (e.g. latent firm characteristics) would have to account for at least $7,1\%$ of the residual variation (in the outcome and treatment) to reduce the lower bound on the effect to zero. Including estimation uncertainty unobserved confounders still have to explain $5.3\%$ of the residual variation to render the effect estimate insignificant.



In [ ]:
dml_plr_obj.sensitivity_plot()

### Sensitivity Analysis with IRM

The sensitivity analysis with the IRM model is basically analogous.



In [ ]:
np.random.seed(42)
dml_irm_obj = dml.DoubleMLIRM(data_dml,
                              ml_g = learner_l,
                              ml_m = learner_m,
                              n_folds = 5)
dml_irm_obj.fit()

print(dml_irm_obj)

In [ ]:
dml_irm_obj.sensitivity_analysis(cf_y=0.04, cf_d=0.03)
print(dml_irm_obj.sensitivity_summary)

In [ ]:
dml_irm_obj.sensitivity_plot()

In [ ]:
fig = dml_irm_obj.sensitivity_plot()

fig.update_layout(
    autosize=False,
    width=500,
    height=500)

fig